<a href="https://colab.research.google.com/github/MatthewYancey/GANime/blob/master/src/process_frames.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Processing

This notebook takes the video files in a folder and saves the individual frames.

## Imports and Parameters

In [1]:
import os
import glob
import cv2
import shutil
from google.colab import drive
import zipfile

In [7]:
# parameters
drive.mount('/content/gdrive')
VIDEO_PATH = '/content/gdrive/My Drive/repos/GANime/data_raw/pokemon/'
TEST_PATH = '/content/gdrive/My Drive/repos/GANime/data_raw/pokemon/'
FRAME_PATH = '/content/frames/'
ZIP_FOLDER = '/content/gdrive/My Drive/repos/GANime/data_out/'

IMAGE_WIDTH = 456
IMAGE_HEIGHT = 288
SINGLE_SIDE = 64 # number of pixels to block out for training

FRAME_SECONDS_SKIP = 0.5
SKIP_SECONDS_BEGINNING = 120
SKIP_SECONDS_END = 120

TRAIN_SPLIT = 0.70
VAL_SPLIT = 0.20
TEST_SPLIT = 0.10

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Video loop and frame saving

In [3]:
def save_frames(video_list, zip_path, four_by_three=False):

    try:
        shutil.rmtree(FRAME_PATH)
    except FileNotFoundError:
        pass
    
    os.mkdir(FRAME_PATH)

    frame_count = 0
    for f in video_list:
        print(f)
        vidcap = cv2.VideoCapture(f)
        video_length = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT) / 24)
        success, image = vidcap.read()

        # loops through and save the frames
        while success:
            current_frame = vidcap.get(cv2.CAP_PROP_POS_FRAMES)

            # skips the intro, outtro, and every 24 frames
            if four_by_three:
                if current_frame % (FRAME_SECONDS_SKIP * 24) == 0:                    
                    # resizes and save the frame
                    image = cv2.resize(image, (IMAGE_WIDTH - (2 * SINGLE_SIDE), IMAGE_HEIGHT))
                    cv2.imwrite(f'{FRAME_PATH}{frame_count}.jpg', image)
                    frame_count += 1

            else:
                if (current_frame >= (SKIP_SECONDS_BEGINNING * 24) and current_frame <= (video_length - SKIP_SECONDS_END) * 24 and current_frame % (FRAME_SECONDS_SKIP * 24) == 0):
                    # resizes and save the frame
                    image = cv2.resize(image, (IMAGE_WIDTH, IMAGE_HEIGHT))
                    cv2.imwrite(f'{FRAME_PATH}{frame_count}.jpg', image)
                    frame_count += 1

            # loop to the next frame
            success, image = vidcap.read()
                
        print(f'Number of images saved: {frame_count}')

    shutil.make_archive(zip_path, 'zip', FRAME_PATH)
    print(f'Saved zip {zip_path}')

In [9]:
VAL_SPLIT / TEST_SPLIT

2.0

In [11]:
# keeps one video out for testing
video_files = glob.glob(VIDEO_PATH + '*')
video_files.sort()
train_files = video_files[:int(len(video_files) * TRAIN_SPLIT):]
video_files = [v for v in video_files if v not in train_files]
val_files = video_files[:int(len(video_files) * (VAL_SPLIT / (VAL_SPLIT + TEST_SPLIT)))]
video_files = [v for v in video_files if v not in val_files]
test_files = video_files

print(f'Train videos: {len(train_files)}')
print(f'Validate videos: {len(val_files)}')
print(f'Test videos: {len(test_files)}')

# saves a train an test zip file
save_frames(test_files, ZIP_FOLDER + 'test')
save_frames(val_files, ZIP_FOLDER + 'validate')
save_frames(train_files, ZIP_FOLDER + 'train')

Train videos: 7
Validate videos: 2
Test videos: 1
